In [1]:
import viz
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def get_id(model, dataset):
    df = pd.read_csv('./experiments/results.txt', names=range(8))
    df[1] = df[1].str.strip()
    df[3] = df[3].astype(str).astype(float)
    id_model = df[(df[0]==model)&(df[1]==dataset)&(df[7]==1)][df[3]==df[3].max()][2]
    print(id_model)
    return model, dataset, id_model

In [3]:
DATASET = 'arrhythmia'
MODEL = 'ann'
OOD = True

In [4]:
ann_url = viz.get_model_url('ann', DATASET, '1619433205')
ann_args = viz.load_args(ann_url)
ann_args.n_ood = 10
x_train, x_val, x_test, y_train, y_val, y_test, ood_x, ood_y = viz.load_data(ann_args)
ann_model = viz.load_model(ann_url, x_train.shape[1], y_train.shape[1], ann_args)

arrhythmia

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x13d77ee90> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x13e52e9d0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x13deaf7d0> and <tensorflow.python.keras.layers.core.Dense object at 0x13d77ee90>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x13ea50250> and <tensorflow.python.keras.layers.core.Dense object at 0x13deaf7d0>).


/Users/JJ/Desktop/ANN-Calibration/viz.py:103: RuntimeWarning: invalid value encountered in true_divide
  x_train = (x_train-n_mean)/n_std
/Users/JJ/Desktop/ANN-Calibration/viz.py:104: RuntimeWarning: divide by zero encountered in true_divide
  x_val = (x_val-n_mean)/n_std
/Users/JJ/Desktop/ANN-Calibration/viz.py:104: RuntimeWarning: invalid value encountered in true_divide
  x_val = (x_val-n_mean)/n_std
/Users/JJ/Desktop/ANN-Calibration/viz.py:105: RuntimeWarning: divide by zero encountered in true_divide
  x_test = (x_test-n_mean)/n_std
/Users/JJ/Desktop/ANN-Calibration/viz.py:105: RuntimeWarning: invalid value encountered in true_divide
  x_test = (x_test-n_mean)/n_std


ValueError: Tensor's shape (128, 10) is not compatible with supplied shape (128, 3)

In [ ]:
jehm_url = viz.get_model_url('jehm', DATASET, '1619406950')
jehm_args = viz.load_args(jehm_url)
jehm_model = viz.load_model(jehm_url, x_train.shape[1], y_train.shape[1], jehm_args)

In [ ]:
jem_url = viz.get_model_url('jem', DATASET, '1616778501')
jem_args = viz.load_args(jem_url)
jem_model = viz.load_model(jem_url, x_train.shape[1], y_train.shape[1], jem_args)

In [ ]:
# Best one thus far 1618054012
# gen ood closest to true ood 1618055607
# gen closes to true ood with not as many o_n steps 1618062683
# Tested with Yining 1617986270

jehmo_url = viz.get_model_url('jehmo', DATASET, '1619408329')
jehmo_args = viz.load_args(jehmo_url)
jehmo_args.n_ood = 10
jehmo_model = viz.load_model(jehmo_url, x_train.shape[1], y_train.shape[1], jehmo_args)

In [ ]:
jehmo_model.n_epochs

In [ ]:
#jemo_url = viz.get_model_url('jemo', DATASET, '1615528859')
#jemo_args = viz.load_args(jemo_url)
#jemo_model = viz.load_model(jemo_url, x_train.shape[1], y_train.shape[1], jemo_args)

In [ ]:
plt.rcParams["figure.figsize"] = (20,3)
def leave_cvx_hull(model_list, x, y):
    fig, axs = plt.subplots(1, 3)
    fig.suptitle(f'Leaving cvx hull {DATASET}')
    for i, y_lbl in enumerate(['accuracy', 'confidence', 'entropy']):
        for model in model_list:
            y_list = []
            for u in range(0, 1000, 100):
                y_plot=0
                t = np.random.uniform(-1, 1, size=x.shape)
                py_x = tf.nn.softmax(model(x + u*t))
                if y_lbl=='entropy':
                    y_plot = -tf.reduce_mean(tf.reduce_sum(py_x*tf.math.log(py_x), axis=1))
                if y_lbl=='accuracy':
                    y_plot = tf.reduce_mean(tf.cast(
                        tf.argmax(py_x, axis=1)==tf.argmax(y, axis=1), tf.float32))
                if y_lbl=='confidence':
                    y_plot = tf.reduce_mean(tf.reduce_max(py_x, 1))
                y_list.append(y_plot.numpy())
            axs[i].plot(np.array(y_list), label=model.name)
        axs[i].set(xlabel='perturbation', ylabel=y_lbl+' (mean)')
        axs[i].legend()

In [ ]:
leave_cvx_hull([ann_model, jehmo_model], x_train, y_train)

In [ ]:
def confidence_plot(models, x, xo):
    for model in models:
        p_in = tf.reduce_max(tf.nn.softmax(model(x)), axis=1)
        p_out = tf.reduce_max(tf.nn.softmax(model(xo)), axis=1)
        plt.ylabel('Frequency')
        plt.xlabel(f'Confidence {DATASET}')
        plt.xlim([0, 1])
        plt.hist(p_in.numpy(), bins=20, label=f'In {model.name}', alpha=.5)
        plt.hist(p_out.numpy(), bins=20, label=f'Out {model.name}', alpha=.5)
        plt.legend()

In [ ]:
xgo = jehmo_model.sample_ood(x_train)
confidence_plot([jehm_model], x_train, xgo)

In [ ]:
confidence_plot([jehm_model], x_test, ood_x)

In [ ]:
confidence_plot([jehmo_model], x_train, ood_x)

In [ ]:
confidence_plot([ann_model], x_train, ood_x)

In [ ]:
confidence_plot([jehmo_model], xgo, ood_x)

In [ ]:
def analyze_features(model, x, xo, idx):
    plt.figure()
    plt.hist(x[:, idx], bins=20, color='blue', label='sample', alpha=.5)
    plt.hist(xo[:, idx], bins=20, color='red', label='ood', alpha=.5)
    if model.name in ['JEM', 'JEHM']:
        xgo = model.sample_ood(x)
        plt.hist(xgo[:, idx].numpy(), bins=20, color='green', label='gen_o', alpha=.5)
    plt.legend()
    plt.show()

In [ ]:
for i in range(4):
    analyze_features(jehmo_model, x_train, ood_x, i)

In [ ]:
def calibration_plot(model, x, y, ece):
    py_x = tf.nn.softmax(model(x))
    p = tf.reduce_max(py_x, axis=1).numpy()
    
    hat_y = tf.argmax(py_x, axis=1)
    y = tf.argmax(y, axis=1)
    acc = tf.cast(hat_y==y, tf.float32).numpy()
    
    acc_list = []
    p_list = []
    for i in np.arange(0,1,.1):
        idx_u = p<=i+.1
        idx_l = i<=p
        idx = idx_l*idx_u
        if p[idx]!=[]:
            p_list.append(np.mean(p[idx]))
            acc_list.append(np.mean(acc[idx]))
        else:
            p_list.append(0)
            acc_list.append(0)
    print(p_list)
    print(acc_list)
    plt.title(f'Calibration {model.name}: {ece}')
    plt.ylabel('Frequency')
    plt.xlabel('ACC/Conf')
    plt.xlim([0, 1])
    plt.bar(np.arange(0,1,.1), height=acc_list, width=.1,
            color='blue', label='accuracy', alpha=.75)
    plt.bar(np.arange(0,1,.1), height=p_list, width=.1,
            color='red', label='confidence', alpha=.75)
    plt.legend()

In [ ]:
calibration_plot(ann_model, x_test, y_test, 0)

In [ ]:
calibration_plot(jehmo_model, x_test, y_test, 0)

In [ ]:
calibration_plot(jem_model, x_test, y_test, 0)

In [ ]:
calibration_plot(jehmo_model, np.concatenate([x_test, ood_x], axis=0), np.concatenate([y_test, ood_y], axis=0), 0)

In [ ]:
calibration_plot(ann_model, np.concatenate([x_test, ood_x], axis=0), np.concatenate([y_test, ood_y], axis=0), 0)

In [ ]:
calibration_plot(jem_model, np.concatenate([x_test, ood_x], axis=0), np.concatenate([y_test, ood_y], axis=0), 0)

In [ ]:
loss = ann_model.evaluate(x_test, y_test, return_dict=True)

In [ ]:
loss

In [ ]:
import umap

#reduce = umap.UMAP()
#z = reduce.fit_transform(np.concatenate([x_train, ood_x], axis=0))


In [ ]:
#import umap
ood_reduce = umap.UMAP()
ood_z = ood_reduce.fit_transform(np.concatenate([x_train, ood_x, xgo.numpy()], axis=0))

In [ ]:
import seaborn as sns
py_x = tf.nn.softmax(jehmo_model(np.concatenate([x_train, ood_x, xgo])))
p = tf.reduce_max(py_x, axis=1).numpy()
markers = ['in']*len(x_train) + ['out']*len(ood_x) + ['gen']*len(xgo.numpy())
plt.figure(figsize=[12,12])
sns.scatterplot(*ood_z.T, c=p, style=markers, cmap='Reds', s=100)
plt.colorbar()
#plt.show()